## 1. 대학 데이터셋을 사용해 소수 인종이 다수인 주 찾기

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
college = pd.read_csv('data/college.csv', index_col='INSTNM')
college.head()

In [ ]:
grouped = college.groupby('STABBR')

In [ ]:
def check_m(df, threshold):
    m_pct = 1 - df['UGDS_WHITE']
    total_m = (df['UGDS'] * m_pct).sum()
    total_ugds = df['UGDS'].sum()
    total_m_pct = total_m / total_ugds
    return total_m_pct > threshold

In [ ]:
c_filter = grouped.filter(check_m, threshold=0.5)
c_filter.head()

In [ ]:
college.shape

In [ ]:
c_filter.shape

In [ ]:
c_filter['STABBR'].nunique()

## 2. 체중 감량 내기의 승자 찾기

In [ ]:
weight_loss = pd.read_csv('data/weight_loss.csv')
weight_loss.query("Month == 'Jan'")

In [ ]:
# 무게감량 구하기
# (현재무게 - 처음무게)/처음무게
def find_perc_loss(s):
    return (s - s.iloc[0]) / s.iloc[0] 

In [ ]:
bob_jan = weight_loss.query("Name=='Bob' and Month=='Jan'")
bob_jan.head()

In [ ]:
bob_jan['Weight'].iloc[0]

In [ ]:
(bob_jan['Weight'] - bob_jan['Weight'].iloc[0])/bob_jan['Weight'].iloc[0]

In [ ]:
find_perc_loss(bob_jan['Weight'])

In [ ]:
pcnt_loss = weight_loss.groupby(['Name','Month'])['Weight'].transform(find_perc_loss)
pcnt_loss.head()

In [ ]:
weight_loss['Perc Weight Loss'] = pcnt_loss

In [ ]:
weight_loss.head()

In [ ]:
weight_loss.query("Name=='Bob' and Month in ['Jan','Feb']")

In [ ]:
week4 = weight_loss.query('Week == "Week 4"')
week4

In [ ]:
winner = week4.pivot(index='Month', columns='Name', values='Perc Weight Loss')
winner

In [ ]:
# np.where :  조건에 맞는 값을 특정 다른 값으로 변환하기
# np.where(조건, 조건에 맞을 때 값, 조건과 다를 때 값)
x = np.array([5, 4, 3, 2, 1, 0])
np.where(x >= 3, 3, x)

In [ ]:
winner['Winner'] = np.where(winner['Amy']<winner['Bob'], 'Amy','Bob')

In [ ]:
winner['Winner']

In [ ]:
winner.style.highlight_min(axis=1)

In [ ]:
winner.Winner.value_counts()

## 3. 주별 가중 평균 SAT 점수 계산

In [ ]:
college.head()

In [ ]:
# 아래에서 5개 볼때
college.tail()

In [ ]:
subset = ['UGDS', 'SATMTMID']
college2 = college.dropna(subset=subset)

In [ ]:
college2.tail()

In [ ]:
college.shape, college2.shape

In [ ]:
# UGDS : 학부생 수, SATMTMID : SAT 수학 점수 중앙값
def weight_math_average(df): # 수학평균
    weight_math = df['UGDS'] * df['SATMTMID']
    return int(weight_math.sum() / df['UGDS'].sum())

In [ ]:
# STABBR : 대학이 위치한 주(state)명
# apply :  각 열(또는 행)을 반복하여 함수를 적용
college.groupby('STABBR').apply(weight_math_average).head()

# 객체 합치기

## 1. concat

### 1.1. Multiple Data Loading

In [ ]:
df1 = pd.read_csv('data/stocks_2016.csv')
df1

In [ ]:
df2 = pd.read_csv('data/stocks_2017.csv')
df2

In [ ]:
df3 = pd.read_csv('data/stocks_2018.csv')
df3

In [ ]:
years = 2016, 2017, 2018
type(years)

In [ ]:
stock_tables = [pd.read_csv('data/stocks_{}.csv'.format(year), index_col='Symbol') for year in years]
stock_tables

In [ ]:
s_2016, s_2017, s_2018 = stock_tables
s_2017

### 1.2. 수직 병합

In [ ]:
pd.concat(stock_tables)

In [ ]:
pd.concat(stock_tables, keys=[2016, 2017, 2018])

### 1.3. 수평 병합

In [ ]:
pd.concat(stock_tables, axis='columns')

In [ ]:
# zip 함수는 동일한 갯수의 요소값을 갖는 시퀀스 자료형을 묶어주는 역할
a = [1,2,3,4,5]
b = ['a','b','c','d','e']
 
for x,y in zip (a,b):
  print (x,y)

In [ ]:
pd.concat(dict(zip(years,stock_tables)), axis='columns')

## 2. join

In [ ]:
# lsuffix : 왼쪽 프레임의 겹치는 열에서 사용할 접미어입니다.
# rsuffix : 오른쪽 프레임의 겹치는 열에서 사용할 접미어입니다.
s_2016.join(s_2017, lsuffix='_2016', rsuffix='_2017')

In [ ]:
s_2016.join(s_2017, lsuffix='_2016', rsuffix='_2017', how='outer')

In [ ]:
s_2017.add_suffix('_2017')

In [ ]:
other = [s_2017.add_suffix('_2017'), s_2018.add_suffix('_2018')]
s_2016.add_suffix('_2016').join(other, how='outer')

## 3. Merge

In [ ]:
s_2016

In [ ]:
s_2017

In [ ]:
s_2016.merge(s_2017, left_index=True, right_index=True)

In [ ]:
step1 = s_2016.merge(s_2017, left_index=True, right_index=True, how='outer', suffixes=('_2016','_2017'))
step1

In [ ]:
s_merge = step1.merge(s_2018.add_suffix('_2018'), left_index=True, right_index=True, how='outer')
s_merge

## 4. 병합의 활용

In [ ]:
names = ['prices', 'transactions']
food_tables = [pd.read_csv('data/food_{}.csv'.format(name)) for name in names]
food_prices, food_tr = food_tables

In [ ]:
food_prices

In [ ]:
food_tr

In [ ]:
food_tr.merge(food_prices, on=['item','store'])

In [ ]:
food_tr.merge(food_prices.query('Date==2017'), how='left')

In [ ]:
f_join = food_prices.query('Date==2017').set_index(['item','store'])

In [ ]:
food_tr.join(f_join, on=['item','store'])

In [ ]:
pd.concat([food_tr.set_index(['item','store']), food_prices.set_index(['item','store'])], axis='columns')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid', palette='muted')
%matplotlib inline

In [ ]:
data = pd.read_csv('data/breast_cancer.csv')

**데이터 확인**

In [ ]:
data.head()

In [ ]:
data.info()

- ID는 분석대상이 아님
- 목표변수: Diagnosis
- Unnamed: 32 열은 모두 NULL값이므로 제거가 필요

**필요 없는 변수 제거 및 x, y 분할**

In [ ]:
col = data.columns
col

In [ ]:
del_col = ['Unnamed: 32', 'id', 'diagnosis']
y = data.diagnosis
x = data.drop(del_col, axis=1)
x.head()

**데이터 살펴보기**

In [ ]:
ax = sns.countplot(y, label='count')
B, M = y.value_counts()
print('양성 :', B)
print('음성 :', M)

In [ ]:
x.describe()

In [ ]:
#!pip install pandas_profiling
import pandas_profiling as pp

In [ ]:
pp.ProfileReport(data)

**데이터 시각화-1 : Violinplot**

In [ ]:
sns.set(style='whitegrid', palette='muted')
graph_data = (x - x.mean()) / (x.std())
graph_data = pd.concat([y, graph_data.iloc[:,0:10]], axis=1)
graph_data = pd.melt(graph_data, id_vars='diagnosis',var_name='features', value_name='value')
plt.figure(figsize=(10,10))
sns.violinplot(x='features', y='value', hue='diagnosis', data=graph_data, split=True, inner='quart')
plt.xticks(rotation=90);

**데이터 시각화-2 : Box Plot**

In [ ]:
graph_data = (x - x.mean()) / (x.std())
graph_data = pd.concat([y, graph_data.iloc[:,20:31]], axis=1)
graph_data = pd.melt(graph_data, id_vars='diagnosis',var_name='features', value_name='value')
plt.figure(figsize=(10,10))
sns.boxplot(x='features', y='value', hue='diagnosis', data=graph_data)
plt.xticks(rotation=90);

**데이터 시각화-3 : Swarm Plot**

In [ ]:
plt.figure(figsize=(10,10))
sns.swarmplot(x='features', y='value', hue='diagnosis', data=graph_data)
plt.xticks(rotation=90);

**변수간 상관관계 파악 1 - 두변수간 상관 관계**

In [ ]:
sns.jointplot(x.loc[:, 'concavity_worst'], x.loc[:, 'concave points_worst'], kind='regg');

**변수간 상관관계 파악 2 - 세 변수 이상 상관 관계**

In [ ]:
df = x.loc[:, ['radius_worst','perimeter_worst','area_worst']]
g = sns.PairGrid(df, diag_sharey=False)
g.map_lower(sns.kdeplot)
g.map_upper(plt.scatter)
g.map_diag(sns.kdeplot);

**변수간 상관관계 파악 3 - HeatMap**

In [ ]:
plt.figure(figsize=(18,18))
sns.heatmap(x.corr(), annot=True, linewidth=.5, fmt='.1f');